In [148]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# Set the Data source connection string. This is where the Arxiv's 9.8k PDFs are. 
# You can change it and use your own data if you wish
DATASOURCE_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=snworkshopstorageaccount;AccountKey=zZqx5QrSv6kgn9mkknZhuRWuQUKoAwosmzlFnofBPc3PRP87DFr+MCgPFnKVfZGjLMSlYolIfeJZ+AStuN4uvw==;EndpointSuffix=core.windows.net"
DATASOURCE_SAS_TOKEN = "sp=r&st=2023-06-27T15:27:20Z&se=2023-06-30T23:27:20Z&spr=https&sv=2022-11-02&sr=c&sig=kawBpMFvouMS31oi3c5obdIdlL34XAbVdBShDDEHL6s%3D"
BLOB_CONTAINER_NAME = "workshop-cognitivesearch"

In [149]:
# # Define the names for the data source, skillset, index and indexer for SharePoint Online
datasource_name = "cogsrch-datasource-files-sp-ananya"
skillset_name = "cogsrch-skillset-files-sp-ananya"
index_name = "cogsrch-index-files-sp-ananya"
indexer_name = "cogsrch-indexer-files-sp-ananya"

In [150]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
AZURE_SEARCH_API_VERSION = '2020-06-30-Preview'
params = {'api-version': AZURE_SEARCH_API_VERSION}

In [151]:
#Create SharePoint Online Data Source

# POST https://[service name].search.windows.net/datasources?api-version=2020-06-30-Preview
# Content-Type: application/json
# api-key: [admin key]

datasource_payload = {
    "name" : datasource_name,
    "type" : "sharepoint",
    "credentials" : { "connectionString" : "SharePointOnlineEndpoint=https://sleepnumberdevtenant.sharepoint.com/sites/sleeplife;ApplicationId=9687bba1-a844-4600-9e19-bafeb9af3b13;TenantId=faf621de-fda8-4938-af5f-804dc43d407d" },
    "container" : {  "name" : "useQuery", "query" : "includeLibrariesInSite=https://sleepnumberdevtenant.sharepoint.com/sites/sleeplife/" }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)
print(r.text)

201
True
{"@odata.context":"https://cog-search-bimbirzg7bdhi.search.windows.net/$metadata#datasources/$entity","@odata.etag":"\"0x8DB775E0428138A\"","name":"cogsrch-datasource-files-sp-ananya","description":null,"type":"sharepoint","subtype":null,"credentials":{"connectionString":null},"container":{"name":"useQuery","query":"includeLibrariesInSite=https://sleepnumberdevtenant.sharepoint.com/sites/sleeplife/"},"dataChangeDetectionPolicy":null,"dataDeletionDetectionPolicy":null,"encryptionKey":null}


In [141]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description": "Extract entities, detect language and extract key-phrases",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
            "description": "Extract text (plain and structured) from image.",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "detectOrientation": True,
            "inputs": [
                {
                  "name": "image",
                  "source": "/document/normalized_images/*"
                }
            ],
                "outputs": [
                {
                  "name": "text",
                  "targetName" : "images_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
            "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field. This is useful for PDF and other file formats that supported embedded images.",
            "context": "/document",
            "insertPreTag": " ",
            "insertPostTag": " ",
            "inputs": [
                {
                  "name":"text", "source": "/document/content"
                },
                {
                  "name": "itemsToInsert", "source": "/document/normalized_images/*/images_text"
                },
                {
                  "name":"offsets", "source": "/document/normalized_images/*/contentOffset"
                }
            ],
            "outputs": [
                {
                  "name": "mergedText", 
                  "targetName" : "merged_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
            "context": "/document",
            "description": "If you have multilingual content, adding a language code is useful for filtering",
            "inputs": [
                {
                  "name": "text",
                  "source": "/document/content"
                }
            ],
            "outputs": [
                {
                  "name": "languageCode",
                  "targetName": "language"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",
            "maximumPageLength": 5000, # 5000 is default
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/content"
                },
                {
                    "name": "languageCode",
                    "source": "/document/language"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "pages"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
            "context": "/document/pages/*",
            "maxKeyPhraseCount": 2,
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/pages/*"
                },
                {
                    "name": "languageCode",
                    "source": "/document/language"
                }
            ],
            "outputs": [
                {
                    "name": "keyPhrases",
                    "targetName": "keyPhrases"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.V3.EntityRecognitionSkill",
            "context": "/document/pages/*",
            "categories": ["Person", "Location", "Organization", "DateTime", "URL", "Email"],
            "minimumPrecision": 0.5, 
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text", 
                    "source":"/document/pages/*"
                },
                {
                    "name": "languageCode",
                    "source": "/document/language"
                }
            ],
            "outputs": [
                {
                    "name": "persons", 
                    "targetName": "persons"
                },
                {
                    "name": "locations", 
                    "targetName": "locations"
                },
                {
                    "name": "organizations", 
                    "targetName": "organizations"
                },
                {
                    "name": "dateTimes", 
                    "targetName": "dateTimes"
                },
                {
                    "name": "urls", 
                    "targetName": "urls"
                },
                {
                    "name": "emails", 
                    "targetName": "emails"
                }
            ]
        }
    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


In [152]:
# Create an index
# Queries operate over the searchable fields and filterable fields in the index
index_payload = {
    "name" : index_name,
    "fields": [
        { "name": "id", "type": "Edm.String", "key": "true", "searchable": "false" },
        { "name": "metadata_spo_item_name", "type": "Edm.String", "key": "false", "searchable": "true", "filterable": "false", "sortable": "false", "facetable": "false" },
        { "name": "metadata_spo_item_path", "type": "Edm.String", "key": "false", "searchable": "false", "filterable": "false", "sortable": "false", "facetable": "false" },
        { "name": "metadata_spo_item_content_type", "type": "Edm.String", "key": "false", "searchable": "false", "filterable": "true", "sortable": "false", "facetable": "true" },
        { "name": "metadata_spo_item_last_modified", "type": "Edm.DateTimeOffset", "key": "false", "searchable": "false", "filterable": "false", "sortable": "true", "facetable": "false" },
        { "name": "metadata_spo_item_size", "type": "Edm.Int64", "key": "false", "searchable": "false", "filterable": "false", "sortable": "false", "facetable": "false" },
        { "name": "content", "type": "Edm.String", "searchable": "true", "filterable": "false", "sortable": "false", "facetable": "false" }
    ]
    # ,"semantic": {
    #   "configurations": [
    #     {
    #       "name": "my-semantic-config",
    #       "prioritizedFields": {
    #         "titleField": 
    #             {
    #                 "fieldName": "metadata_spo_item_name"
    #             },
    #         "prioritizedContentFields": [
    #             {
    #                 "fieldName": "content"
    #             }
    #             ]
    #       }
    #     }
    #   ]
    # }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)
print(r.text)

201
True
{"@odata.context":"https://cog-search-bimbirzg7bdhi.search.windows.net/$metadata#indexes/$entity","@odata.etag":"\"0x8DB775E0A16AF49\"","name":"cogsrch-index-files-sp-ananya","defaultScoringProfile":null,"fields":[{"name":"id","type":"Edm.String","searchable":false,"filterable":true,"retrievable":true,"sortable":true,"facetable":true,"key":true,"indexAnalyzer":null,"searchAnalyzer":null,"analyzer":null,"normalizer":null,"synonymMaps":[]},{"name":"metadata_spo_item_name","type":"Edm.String","searchable":true,"filterable":false,"retrievable":true,"sortable":false,"facetable":false,"key":false,"indexAnalyzer":null,"searchAnalyzer":null,"analyzer":null,"normalizer":null,"synonymMaps":[]},{"name":"metadata_spo_item_path","type":"Edm.String","searchable":false,"filterable":false,"retrievable":true,"sortable":false,"facetable":false,"key":false,"indexAnalyzer":null,"searchAnalyzer":null,"analyzer":null,"normalizer":null,"synonymMaps":[]},{"name":"metadata_spo_item_content_type","type

In [153]:
# POST https://[service name].search.windows.net/indexers?api-version=2020-06-30-Preview

# Create an indexer
indexer_payload = {
    "name" : indexer_name,
    "dataSourceName" : datasource_name,
    "targetIndexName" : index_name,
    #"skillsetName": skillset_name,
    "schedule" : { "interval" : "PT2H"},
    "parameters": {
        "configuration": {
            "dataToExtract": "contentAndMetadata"
           # ,"indexedFileNameExtensions" : ".pdf, .docx, .aspx"
           # ,"excludedFileNameExtensions" : ".png, .jpg"
          
      }
    },
    "schedule" : { },
    #"skillsetName" : skillset_name,
    "fieldMappings" : [
        {
            "sourceFieldName" : "metadata_spo_site_library_item_id",
            "targetFieldName" : "id",
            "mappingFunction" : {
                "name" : "base64Encode"
            }
        }
    ]
  
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)
print(r.text)

201
True
{"@odata.context":"https://cog-search-bimbirzg7bdhi.search.windows.net/$metadata#indexers/$entity","@odata.etag":"\"0x8DB775E126E2915\"","name":"cogsrch-indexer-files-sp-ananya","description":null,"dataSourceName":"cogsrch-datasource-files-sp-ananya","skillsetName":null,"targetIndexName":"cogsrch-index-files-sp-ananya","disabled":null,"schedule":null,"parameters":{"batchSize":null,"maxFailedItems":null,"maxFailedItemsPerBatch":null,"base64EncodeKeys":null,"configuration":{"dataToExtract":"contentAndMetadata"}},"fieldMappings":[{"sourceFieldName":"metadata_spo_site_library_item_id","targetFieldName":"id","mappingFunction":{"name":"base64Encode","parameters":null}}],"outputFieldMappings":[],"cache":null,"encryptionKey":null}


In [145]:
# Optionally, get indexer status to confirm that it's running
r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                 "/status", headers=headers, params=params)
# pprint(json.dumps(r.json(), indent=1))
print(r.status_code)
# print("Status:",r.json().get('lastResult').get('status'))
# print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
print(r.text)

200
{"@odata.context":"https://cog-search-bimbirzg7bdhi.search.windows.net/$metadata#Microsoft.Azure.Search.V2020_06_30_Preview.IndexerExecutionInfo","name":"cogsrch-indexer-files-sp-ananya","status":"running","lastResult":{"status":"inProgress","statusDetail":null,"errorMessage":null,"startTime":"2023-06-27T22:20:43.296Z","endTime":null,"itemsProcessed":0,"itemsFailed":0,"initialTrackingState":null,"finalTrackingState":null,"mode":"indexingAllDocs","errors":[],"warnings":[],"metrics":null},"executionHistory":[],"limits":{"maxRunTime":"P1D","maxDocumentExtractionSize":134217728,"maxDocumentContentCharactersToExtract":4194304},"currentState":{"mode":"indexingAllDocs","allDocsInitialTrackingState":null,"allDocsFinalTrackingState":null,"resetDocsInitialTrackingState":null,"resetDocsFinalTrackingState":null,"resetDocumentKeys":[],"resetDatasourceDocumentIds":[]}}


# 
****
References: https://learn.microsoft.com/en-us/azure/search/search-howto-index-sharepoint-online
****